<a href="https://colab.research.google.com/github/RaIvPa/Improving-BERT-for-Biomedical-QA/blob/main/practice/BERT_base_sentence_classification_with_BioASQ_preprocessing_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets transformers[sentencepiece]
!pip install py-rouge

import transformers
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from datasets import load_dataset, load_metric
import numpy as np
import csv
from csv import reader
import json
from rouge import Rouge
import nltk

     |████████████████████████████████| 2.8 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 39.8 MB/s 
     |████████████████████████████████| 636 kB 45.9 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 38.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 4.1 MB/s 
     |████████████████████████████████| 243 kB 47.4 MB/s 
     |████████████████████████████████| 119 kB 51.1 MB/s 
     |████████████████████████████████| 1.1 MB 33.0 MB/s 
     |████████████████████████████████| 56 kB 2.4 MB/s 


In [2]:
metric = Rouge(metrics=["rouge-n"], max_n=2)

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
# log_model = TFAutoModel.from_pretrained(checkpoint)
# implement top-3

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
def create_training_dataset(csv_name, data_file_name, data_num):  
  csv_file = open(csv_name, 'w')
  data_file = open(data_file_name, 'r')

  data_load = json.load(data_file)
  csv_write = csv.writer(csv_file)
  csv_write.writerow(["question", "snippet", "rouge-2", "label"])

  rouge_score_get = Rouge(metrics=["rouge-n"], max_n=2)
    
  rows = []
  top_n = 5
  question_num = 0
  for question_set in data_load["questions"]:
    if question_num <= data_num:
      sub_rows = []
      question = question_set["body"]
      top = []
      top_id = {}
      snip_num = 0
      chosen = 0
      for snippet in question_set["snippets"]:
          snip_text = snippet["text"]
          rouge_score = rouge_score_get.get_scores(question, snip_text)
          # print(rouge_score)
          rouge_2_f = rouge_score["rouge-2"]["f"]
          top.append(rouge_2_f)
          # top.sort(reverse=True)
          # if check <= 16:
          #   print(top)
          #   check += 1
          #   # print(top_id)
          #   if len(top) == 0:
          #     top.append(rouge_2_f)
          #     top_id[rouge_2_f] = snip_num
          #   elif len(top) < top_n:
          #     top.append(rouge_2_f)
          #     top_id[rouge_2_f] = snip_num
          #   elif len(top) == top_n:
          #     if rouge_2_f >= top[top_n-1]:
          #       top_id.pop(top[top_n-1])
          #       top.pop(top_n-1)
          #       top.append(rouge_2_f)
          #       top_id[rouge_2_f] = snip_num
          sub_rows.append([question, snip_text, rouge_2_f])
          #   snip_num += 1
          #   print(top_id)
        
      top_indx = sorted(range(len(top)), key=lambda i: top[i], reverse=True)[:5]
      # print(top_indx)

      for indx in top_indx:
        sub_rows[indx].append(1)

      for row in sub_rows:
        if len(row) < 4:
          row.append(0)
        # score = row[2]
        # if len(top) < top_n:
        #   row.append(1)
        # elif score >= top[top_n-1] and len(top) == top_n and chosen < top_n:
        #   row.append(1)
        #   chosen += 1
        # else:
        #   row.append(0)


      rows.append(sub_rows)
    question_num += 1

  for final_row in rows:
    csv_write.writerows(final_row)

  csv_file.close()
  data_file.close()

In [4]:
def create_validation_dataset(csv_name, data_file_name, data_num):  
  csv_file = open(csv_name, 'w')
  data_file = open(data_file_name, 'r')

  data_load = json.load(data_file)
  csv_write = csv.writer(csv_file)
  csv_write.writerow(["question", "snippet", "rouge-2", "label"])

  rouge_score_get = Rouge(metrics=["rouge-n"], max_n=2)
    
  rows = []
  top_n = 5
  question_num = 0
  for question_set in data_load["questions"]:
    if question_num > data_num and question_num <= data_num*2:
      sub_rows = []
      question = question_set["body"]
      top = []
      top_id = {}
      snip_num = 0
      chosen = 0
      for snippet in question_set["snippets"]:
          snip_text = snippet["text"]
          rouge_score = rouge_score_get.get_scores(question, snip_text)
          rouge_2_f = rouge_score["rouge-2"]["f"]
          top.append(rouge_2_f)
          # top.sort(reverse=True)
          # if check <= 16:
          #   print(top)
          #   check += 1
          #   # print(top_id)
          #   if len(top) == 0:
          #     top.append(rouge_2_f)
          #     top_id[rouge_2_f] = snip_num
          #   elif len(top) < top_n:
          #     top.append(rouge_2_f)
          #     top_id[rouge_2_f] = snip_num
          #   elif len(top) == top_n:
          #     if rouge_2_f >= top[top_n-1]:
          #       top_id.pop(top[top_n-1])
          #       top.pop(top_n-1)
          #       top.append(rouge_2_f)
          #       top_id[rouge_2_f] = snip_num
          sub_rows.append([question, snip_text, rouge_2_f])
          #   snip_num += 1
          #   print(top_id)
        
      top_indx = sorted(range(len(top)), key=lambda i: top[i], reverse=True)[:5]
      # print(top_indx)

      for indx in top_indx:
        sub_rows[indx].append(1)

      for row in sub_rows:
        if len(row) < 4:
          row.append(0)
        # score = row[2]
        # if len(top) < top_n:
        #   row.append(1)
        # elif score >= top[top_n-1] and len(top) == top_n and chosen < top_n:
        #   row.append(1)
        #   chosen += 1
        # else:
        #   row.append(0)


      rows.append(sub_rows)
    question_num += 1

  for final_row in rows:
    csv_write.writerows(final_row)

  csv_file.close()
  data_file.close()

CSV column format is


*   Question
*   Snippet
*   ROUGE score
*   Label




In [5]:
dataset_split_num = 250
nltk.download("punkt")
create_training_dataset('training_data.csv', '/content/drive/MyDrive/Thesis/training8b.json', dataset_split_num)
create_validation_dataset('validation_data.csv', '/content/drive/MyDrive/Thesis/training8b.json', dataset_split_num)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
datasets = load_dataset('csv', data_files={"train": 'training_data.csv', 
                                           "validation": "validation_data.csv"
                                           })
print(datasets)

Using custom data configuration default-5ec955671521cd9b


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5ec955671521cd9b/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['question', 'snippet', 'rouge-2', 'label'],
        num_rows: 3829
    })
    validation: Dataset({
        features: ['question', 'snippet', 'rouge-2', 'label'],
        num_rows: 3510
    })
})


In [7]:
def tokenize_dataset(dataset):
  encoded = tokenizer(
      dataset["question"],
      dataset["snippet"],
      padding=True,
      truncation=True,
      return_tensors="np",
  )
  return encoded.data

In [8]:
tok_data = {
    split: tokenize_dataset(datasets[split]) for split in datasets.keys()
}

# tok_data = tokenize_dataset(datasets["train"])

In [9]:
print(tok_data)

{'train': {'input_ids': array([[  101,  2003, 28127, ...,     0,     0,     0],
       [  101,  2003, 28127, ...,     0,     0,     0],
       [  101,  2003, 28127, ...,     0,     0,     0],
       ...,
       [  101,  2515,  5255, ...,     0,     0,     0],
       [  101,  2029, 16012, ...,     0,     0,     0],
       [  101,  2029, 16012, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}, 'validation': {'input_ids': array([[ 101, 4013, 2618, ...,    0,    0,    0],
       [ 101, 4013, 2618, ...,    0,    0,    0],
       [ 101, 4013, 2618, ...,    0,    0,    0],

In [10]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

In [11]:
bat_size = 8
epoch_num = 1

train_steps = (len(tok_data["train"]["input_ids"]) // bat_size) * epoch_num
lr_scheduler = PolynomialDecay(
    initial_learning_rate=1e-4,
    end_learning_rate=0.,
    decay_steps=train_steps
    )

In [12]:
from tensorflow.keras.optimizers import Adam

In [13]:
opt = Adam(learning_rate=lr_scheduler)

In [14]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import Input

Implementing linear regression into the system

In [ ]:
new_model_inputs = Input(shape = (322,), dtype='int64')
new_model = (model)(new_model_inputs)
flat_model = Flatten()(new_model[0])
classifier = Dense(units=len(tok_data["train"]), activation="linear")(flat_model) 
fin_model = Model(inputs=new_model_inputs, outputs=classifier)
fin_model.summary()

In [ ]:
from tqdm.keras import TqdmCallback

fin_model.compile(optimizer=opt, loss=MeanSquaredError(), metrics=["accuracy"])

fin_model.fit(
    tok_data["train"],
    np.array(datasets["train"]["label"]),
    validation_data=(
        tok_data["validation"],
        np.array(datasets["validation"]["label"])
    ),
     # validation_split=0.20,
    validation_steps=100,
    validation_batch_size=bat_size,
    batch_size=bat_size,
    verbose=0,
    callbacks=[TqdmCallback(verbose=2)],
    steps_per_epoch=100
    # steps_per_epoch=1 # for debugging
)

In [ ]:
from tqdm.keras import TqdmCallback

model.compile(optimizer=opt, loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

model.fit(
    tok_data["train"],
    np.array(datasets["train"]["label"]),
    validation_data=(
        tok_data["validation"],
        np.array(datasets["validation"]["label"])
    ),
    # validation_split=0.20,
    # validation_steps=100,
    validation_batch_size=bat_size,
    batch_size=bat_size,
    verbose=0,
    callbacks=[TqdmCallback(verbose=2)],
    epochs=epoch_num,
    # steps_per_epoch=100
    # steps_per_epoch=1 # for debugging
)

0epoch [00:00, ?epoch/s]

  0%|          | 0.00/479 [00:00<?, ?batch/s]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

  0%|          | 0.00/479 [00:00<?, ?batch/s]

  0%|          | 0.00/479 [00:00<?, ?batch/s]

  0%|          | 0.00/479 [00:00<?, ?batch/s]

  0%|          | 0.00/479 [00:00<?, ?batch/s]

  0%|          | 0.00/479 [00:00<?, ?batch/s]

  0%|          | 0.00/479 [00:00<?, ?batch/s]

  0%|          | 0.00/479 [00:00<?, ?batch/s]

In [ ]:
!nvidia-smi

Sat Sep  4 16:05:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    59W / 149W |   8360MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Linear Regression - BERT base model (with only training data)

479/479 batches

loss: 4.4516

accuracy: 0.5377

Logistic Regression - BERT base model (with only training data) (accuracy starts to go down past 370)


479/479 batches

loss: 0.6272

accuracy: 0.6929

adding validation data seems to cause problems with the input layer for linear approach (not the case, see below)

loss seems to go nan when adding validation data (problem fixed; len function in training steps wasn't accounting the fact that there were now two datasets in tok_data)

fitting code seems to keep executing past the first 479 steps, but doesn't show more fitting output (might be because of epoch number discrepancies between the scheduler and fit function)

In [ ]:
# for each question:
  # for each snippet:
    # check if it is in top 3 rouge scores
  # concatenate top 3 snippets together to make summary
  # measure rouge-2 f score between summary and ideal answer
# show overall rouge-2 f score between summaries and ideal answers


#todo - make ideal answer scraper

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)
predictions = model.predict(tok_data["validation"])["logits"]
predictions

In [ ]:
highest_predictions = np.argmax(predictions, axis=1)

In [ ]:
highest_predictions.view()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")
metric

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions: Predicted labels, as returned by a model.
    references: Ground truth labels.
    normalize: If False, return the number of correctly classified samples.
        Otherwise, return the fraction of correctly classified samples.
    sample_weight: Sample weights.
Returns:
    accuracy: Accuracy score.
Examples:

    >>> accuracy_metric = datasets.load_metric("accuracy")
    >>> results = accuracy_metric.compute(references=[0, 1], predictions=[0, 1])
    >>> print(results)
    {'accuracy': 1.0}
""", stored examples: 0)

In [ ]:
# metric._get_scores_rouge_n(all_hypothesis=str(highest_predictions), all_references=datasets["validation"]["label"])["rouge-2"]["f"]


metric.compute(predictions=highest_predictions, references=datasets['validation']['label'])

{'accuracy': 0.6621082621082621}

issue with compute function not taking the validation label list due to type mismatch: expected bytes, got int